In [ ]:
from pathlib import Path


import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, hstack, vstack,join
from astropy.coordinates import SkyCoord
import astropy.units as u
import pandas as pd
from scipy.integrate import trapezoid

import desispec.io
from desispec.spectra import stack

from prospect import utilities

In [ ]:
params = {
    "legend.fontsize": "x-large",
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
    "figure.facecolor": "w",
    "xtick.top": True,
    "ytick.right": True,
    "xtick.direction": "in",
    "ytick.direction": "in",
    "font.family": "serif",
    "mathtext.fontset": "dejavuserif"
}
plt.rcParams.update(params)

In [ ]:
spec_base_path = Path("/global/cfs/cdirs/desi/users/raichoor/laelbg/daily/healpix/")
#temporary
spec_base_path = spec_base_path / "tertiary15-thru20221216"


In [ ]:
img_base_path = Path("/global/cfs/cdirs/desi/users/bid13/DESI_II/target_data")
patch = "XMM"
hsc_path = img_base_path / f"HSC_{patch}_I_mag_lim_24.8.fits"

def flux_to_mag(flux):
    return -2.5*np.log10(flux*1e-9) + 8.90

hsc_cat = Table.read(hsc_path).to_pandas()
# extinction corrected mags (extinction is negligible for XMM-LSS)
hsc_cat["mag_i"] = flux_to_mag(hsc_cat["i_cmodel_flux"])-hsc_cat["a_i"]
hsc_cat["mag_r"] = flux_to_mag(hsc_cat["r_cmodel_flux"])-hsc_cat["a_r"]
hsc_cat["mag_z"] = flux_to_mag(hsc_cat["z_cmodel_flux"])-hsc_cat["a_z"]
hsc_cat["mag_g"] = flux_to_mag(hsc_cat["g_cmodel_flux"])-hsc_cat["a_g"]

hsc_cat["mag_g_fiber"] = flux_to_mag(hsc_cat["g_fiber_flux"])-hsc_cat["a_g"]
hsc_cat["mag_i_fiber"] = flux_to_mag(hsc_cat["i_fiber_flux"])-hsc_cat["a_i"]
hsc_cat["mag_r_fiber"] = flux_to_mag(hsc_cat["r_fiber_flux"])-hsc_cat["a_r"]
hsc_cat["mag_z_fiber"] = flux_to_mag(hsc_cat["z_fiber_flux"])-hsc_cat["a_z"]
# hsc_cat["i_fiber_tot_mag"] = flux_to_mag(hsc_cat["i_fiber_tot_flux"])-hsc_cat["a_i"]

## Quality cuts
# valid I-band flux
qmask = np.isfinite(hsc_cat["i_cmodel_flux"]) & (hsc_cat["i_cmodel_flux"]>0)
#cmodel fit not failed
qmask &= (~hsc_cat["i_cmodel_flag"].values)
#General Failure Flag
qmask &= (~hsc_cat["i_sdsscentroid_flag"].values)

# Possible cuts: Bright objects nearby, bad pixels

#star-galaxy separation (is point source in I band)
# mask &= (hsc_cat["i_extendedness_value"]>0)


i_min = 22
i_max = 24.5
mask = (hsc_cat["mag_i"] <i_max) & (hsc_cat["mag_i"] >i_min)
i_fiber_min = 22
i_fiber_max = 25 #for cosmos the limit is 25 whereas for XMM it was set to 24.75
mask &= (hsc_cat["mag_i_fiber"] <i_fiber_max) & (hsc_cat["mag_i_fiber"] >i_fiber_min)


sels_cat = hsc_cat[qmask & mask]
sels_cat = sels_cat.reset_index()

In [ ]:
spec_cat = Table.read(spec_base_path / "merge-tertiary15-thru20221216.fits")

spec_cat = spec_cat[np.isin(spec_cat["TERTIARY_TARGET"], [b"LSST_WLY1_HIP", b"LSST_WLY1_LOP"])]

In [ ]:
hsc_coord = SkyCoord(ra=sels_cat["ra"].values*u.degree, dec=sels_cat["dec"].values*u.degree)

spec_coord = SkyCoord(ra=spec_cat["TARGET_RA"]*u.degree, dec=spec_cat["TARGET_DEC"]*u.degree)
idx, d2d, d3d = spec_coord.match_to_catalog_sky(hsc_coord)

phot_cat = Table.from_pandas(sels_cat.iloc[idx])

In [ ]:
cat = hstack([spec_cat, phot_cat])

In [ ]:
plt.hist(cat['mag_i'], bins=20)
plt.xlabel("i-mag")

In [ ]:
plt.hist(cat['mag_i_fiber'], bins=20)
plt.xlabel("i-fiber-mag")

In [ ]:
plt.hist(cat["Z"], bins=50)
plt.xlabel("Spec-z")

In [ ]:
plt.scatter(cat['mag_i'], cat["DELTACHI2"], marker=".")

plt.xlabel("i_mag")
plt.ylabel("DELTACHI2")
plt.yscale("log")
plt.ylim(0.1,5000)

In [ ]:
plt.scatter(cat['mag_i_fiber'], cat["DELTACHI2"], marker=".")
plt.ylim(-10,500)
plt.xlabel("i_fiber_mag")
plt.ylabel("DELTACHI2")

In [ ]:
cat["EXPTIME"] = 12.15 * cat["TSNR2_LRG"]

In [ ]:
# plt.hist(cat["COADD_EXPTIME"]/3600, bins=20)
plt.hist(cat["EXPTIME"]/3600, bins=100)
plt.xlabel("Exposure time (Hours)")

In [ ]:
plt.hist(cat["DELTACHI2"],range=(0,10000), bins=20)
plt.yscale("log")
plt.xlabel("DELTACHI2")

In [ ]:
plt.scatter(cat["EXPTIME"]/3600, cat["DELTACHI2"], marker=".")
plt.ylim(-10,1000)
plt.xlabel("COADD_EXPTIME")
plt.ylabel("DELTACHI2")

In [ ]:
plt.scatter(cat["mag_i"], cat["COADD_EXPTIME"]/3600, marker=".")

plt.xlabel("i_mag")
plt.ylabel("EXPTIME")

### Create one single dataset

In [ ]:
cat.add_index("TARGETID")

spec_obj_list = []
rr_table_list = []
rr_details_list = []

for f in spec_base_path.glob("coadd*"):
    spectra = desispec.io.read_spectra(f)
    mask = spectra.fibermap["OBJTYPE"] == "TGT"
    mask &= np.isin(spectra.fibermap["TARGETID"], cat["TARGETID"])
    spectra = spectra[mask]
    if spectra.num_spectra() >0:
        for c in ["mag_i", "mag_i_fiber", "EXPTIME"]:
            spectra.fibermap[c] = cat.loc[spectra.fibermap["TARGETID"]][c]
        spec_obj_list.append(spectra)
        
        rr_table = Table.read(str(f).replace("coadd", "redrock"))
        mask = np.isin(rr_table["TARGETID"], cat["TARGETID"])
        rr_table_list.append(rr_table[mask])

        redrock_det_file = str(f).replace("coadd", "rrdetails").replace("fits", "h5")
        rr_details_tab = utilities.match_rrdetails_to_spectra(redrock_det_file, spectra)

        rr_details_list.append(rr_details_tab)
    
    

In [ ]:
rr_table = vstack(rr_table_list)
rr_details_table = vstack(rr_details_list)
spectra = stack(spec_obj_list)

In [ ]:
out_path = Path("/global/cfs/cdirs/desi/users/bid13/DESI_II/pilot_obs")

In [ ]:
desispec.io.write_spectra(out_path / "pilot_spectra.fits", spectra)
rr_table.write(out_path / "pilot_zbest.fits")
rr_details_table.write(out_path / "pilot_redrock_details.fits")
cat.write(out_path / "merged_cat.fits")